In [1]:
import datetime
import os
import time
import shutil
import subprocess
import sys

import torch
import torch.utils.data
from models import detection
from torch.utils.tensorboard import SummaryWriter

from coco_utils import get_coco  # get_coco_kp

from group_by_aspect_ratio import GroupedBatchSampler, create_aspect_ratio_groups
from engine import train_one_epoch, evaluate

import utils

ImportError: dlopen(/Users/sglee/anaconda3/envs/linc-detector/lib/python3.7/site-packages/torchvision/_C.cpython-37m-darwin.so, 0x0002): symbol not found in flat namespace '__ZN6caffe26detail36_typeMetaDataInstance_preallocated_0E'